In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [131]:
pages=5
names_list=[]
company_list=[]
desc_list=[]
skills_l=[]
for page in range(pages):
    url = f'https://hh.ru/search/vacancy?text=Middle+python&from=suggest_post&area=1&page={page}'
    user_agent = {'User-agent': 'Mozilla/5.0'}
    result = requests.get(url, headers=user_agent)
    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'lxml')
        names = soup.find_all('a', attrs={'data-qa': 'serp-item__title', 'class':'serp-item__title'})
        for name in names:
                url = name.attrs.get('href')
                result = requests.get(url, headers=user_agent)
                soup = BeautifulSoup(result.content, 'lxml')
                names = soup.find_all('h1',{'data-qa':'vacancy-title'})
                if names:
                    names = names[0].text
                else:
                    names=None
                company = soup.find('span',{'class':'vacancy-company-name'})
                if company != None:
                    company = company.text
                else:
                    next
                description = soup.find('div',{'class':'g-user-content','data-qa':'vacancy-description'})
                if description:
                    description=description.text
                else:
                    description=description
                skills = soup.find_all('div',{'data-qa':'bloko-tag bloko-tag_inline skills-element'})
                skills_list=[skill.text for skill in skills]
                skills_list = ", ".join(skills_list)
                names_list.append(names)
                company_list.append(company)
                desc_list.append(description)
                skills_l.append(skills_list)

In [134]:
temp = {'names':names_list,'company':company_list,'description':desc_list,'skills':skills_l}
data = pd.DataFrame(temp)
len(data)

100

In [135]:
url = "https://api.hh.ru/vacancies?text=middle python&per_page=100&page=0"

In [136]:
result = requests.get(url)

In [137]:
vacancies = result.json().get('items')

In [138]:
links = []
for i in range(len(vacancies)):
    links.append(vacancies[i]['url'])

In [139]:
names_list=[]
company_list=[]
desc_list=[]
skills_l=[]
for link in links:
    url=link
    result = requests.get(url)
    vacancy = result.json()
    name = vacancy['name']
    emp = vacancy['employer']['name']
    desc = vacancy['description']
    skills =  [i['name'] for i in vacancy['key_skills']]
    skills = ", ".join(skills)
    names_list.append(name)
    company_list.append(emp)
    desc_list.append(desc)
    skills_l.append(skills)

In [140]:
temp = {'names':names_list,'company':company_list,'description':desc_list,'skills':skills_l}
data2 = pd.DataFrame(temp)
data2

,names,company,description,skills
0,QA Engineer/Тестировщик (Удаленно),Professional Gaming Solutions Ltd,<p>Мы ищем Junior/Middle QA тестировщика для р...,"QA, MVT тесты, Тестирование, Автотесты"
1,Python разработчик,TerraLink,<p><em>Компания</em><em> ТерраЛинк </em><em>– ...,
2,QA Engineer (Junior+/ Middle),AMarkets,<p><strong>AMarkets</strong> – международная F...,"Функциональное тестирование, Postman, Тестиров..."
3,Middle Vue разработчик,Sibdev,<p><em>Мы - аккредитованная компания <strong>S...,"Redis, Vue, Vuex, RabbitMQ"
4,Junior/Middle backend разработчик на Fast API ...,Webtronics,<p>Привет! Мы Webtronics. Ищем в команду FastA...,"Git, Redis, PostgreSQL, Python, REST, REST API..."
...,...,...,...,...
95,Middle / Middle+ Python Developer,Holyweb,<p><strong>Holyweb</strong> – сервисная IT-ком...,"PostgreSQL, Python, Kafka, Docker, Django Fram..."
96,Разработчик аналитических систем,Крит,<p>Мы предлагаем работу на крупных проектах у ...,
97,Middle PHP-разработчик,Директ Лайн,<p>Приглашаем в команду <strong>PHP разработчи...,"Laravel, MariaDB, ООП, PHP, PHP7, API, Git, Yi..."
98,Инженер по автоматизации тестирования (Middle),Компания Стрим Лабс,<p>Мы разрабатываем ПО для контроля и монитори...,


In [80]:
import sqlite3

In [81]:
conn = sqlite3.connect('db.db')
cur=conn.cursor()

In [142]:
cur.execute("""
    create table if not exists hh(
        id integer primary key,
        name text,
        company text,
        description text,
        skills text);
""")
conn.commit()

In [143]:
#Запись в бд данных полученных парсингом
for i in range(len(data)):
    cur.execute(
        """
            insert into hh(name, company, description, skills)
            values (?,?,?,?);
        """,data.iloc[i]
        )
    conn.commit()

In [144]:
cur.execute(
"""select * from hh
"""
)
res = cur.fetchall()


In [145]:
cur.execute("""
    create table if not exists hh_api(
        id integer primary key,
        name text,
        company text,
        description text,
        skills text);
""")
conn.commit()

In [146]:
#Запись в бд данных полученных по API
for i in range(len(data2)):
    cur.execute(
        """
            insert into hh(name, company, description, skills)
            values (?,?,?,?);
        """,data2.iloc[i]
        )
    conn.commit()